## objective: To compute features

In [41]:
from data_processing import load_data, get_files, parse_filename, drop_nd_rows
from feature_extraction import extract_fft_features, extract_time_domain_features
from transformations import interpolate_zeros, replace_outliers_with_zeros, replace_outliers
import pandas as pd
import dask
from dask import do
from os.path import split
from dask import delayed, compute
import numpy as np

from os import listdir
import pandas as pd
from os.path import join, split
from data_processing import *
%matplotlib inline

from scipy.stats import shapiro
from scipy.stats.mstats import normaltest
from scipy.stats import iqr
%load_ext autoreload

pd.set_option('display.max_columns',999)
pd.set_option('display.max_rows',999)

from dask.diagnostics import ProgressBar

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
paths =['/Users/crivera5/Documents/NonIntuitProjects/Kaggle/KaggleEEG/train_1',
       '/Users/crivera5/Documents/NonIntuitProjects/Kaggle/KaggleEEG/train_2',
       '/Users/crivera5/Documents/NonIntuitProjects/Kaggle/KaggleEEG/train_3']

In [3]:
path = [paths[1]]
processor = Processor()


res = processor.process_data(path)

In [5]:
res.head()

,Variance 1,Variance 2,Variance 3,Variance 4,Variance 5,Variance 6,Variance 7,Variance 8,Variance 9,Variance 10,...,SpectralEntropy 10,SpectralEntropy 11,SpectralEntropy 12,SpectralEntropy 13,SpectralEntropy 14,SpectralEntropy 15,SpectralEntropy 16,patient,dataset_id,pre_ictal
0,2889.705078,3052.641113,3397.204346,2022.422607,6143.035156,4742.090820,4253.417480,3495.472656,2854.358398,0.280135,...,NaN,-1.304488,-1.263821,-1.302768,-1.193445,-1.299206,-1.339494,2,1007,0
0,2710.423828,4334.229980,3632.799316,2216.874756,7091.765625,4323.705078,4576.989258,3881.434082,2740.354004,0.275762,...,NaN,-1.277472,-1.315704,-1.322317,-1.148335,-1.292326,-1.311846,2,1008,0
0,2371.291992,1558.058594,1582.461914,1332.104736,5459.472656,6591.005371,4970.944824,4074.741211,1575.949219,1275.349854,...,-1.281061,-1.294191,-1.265645,-1.280330,-1.206379,-1.295410,-1.346674,2,100,0
0,1568.713013,1608.974976,1388.828247,2282.002686,3534.720459,2007.592041,1384.285889,1410.421265,764.312561,755.743225,...,-1.420582,-1.368031,-1.451717,-1.422706,-1.387995,-1.478017,-1.483069,2,100,1
0,2675.753174,2772.882812,2939.495361,1802.443726,6281.311035,5143.335938,4026.415771,3052.296387,2303.293701,0.272071,...,NaN,-1.305050,-1.263693,-1.366296,-1.191261,-1.312526,-1.335824,2,1016,0


In [11]:
#save it 
from os.path import join
res.to_csv(join('/Users/crivera5/Documents/NonIntuitProjects/Kaggle/KaggleEEG/', 'data_local', 'patient2.csv'), index = False)

In [12]:
df = pd.read_csv(join('/Users/crivera5/Documents/NonIntuitProjects/Kaggle/KaggleEEG/', 'data_local', 'patient2.csv'))

In [42]:
res.isnull().sum()

Variance 1                         0
Variance 2                         0
Variance 3                         0
Variance 4                         0
Variance 5                         0
Variance 6                         0
Variance 7                         0
Variance 8                         0
Variance 9                         0
Variance 10                        0
Variance 11                        0
Variance 12                        0
Variance 13                        0
Variance 14                        0
Variance 15                        0
Variance 16                        0
Kurtosis 1                         0
Kurtosis 2                         0
Kurtosis 3                         0
Kurtosis 4                         0
Kurtosis 5                         0
Kurtosis 6                         0
Kurtosis 7                         0
Kurtosis 8                         0
Kurtosis 9                         0
Kurtosis 10                        0
Kurtosis 11                        0
K

### Do with all the data

In [46]:
paths =['/Users/crivera5/Documents/NonIntuitProjects/Kaggle/KaggleEEG/train_1',
       '/Users/crivera5/Documents/NonIntuitProjects/Kaggle/KaggleEEG/train_2',
       '/Users/crivera5/Documents/NonIntuitProjects/Kaggle/KaggleEEG/train_3']

res2 = processor.process_data(paths)

In [47]:
res2.head()

,Variance 1,Variance 2,Variance 3,Variance 4,Variance 5,Variance 6,Variance 7,Variance 8,Variance 9,Variance 10,Variance 11,Variance 12,Variance 13,Variance 14,Variance 15,Variance 16,Kurtosis 1,Kurtosis 2,Kurtosis 3,Kurtosis 4,Kurtosis 5,Kurtosis 6,Kurtosis 7,Kurtosis 8,Kurtosis 9,Kurtosis 10,Kurtosis 11,Kurtosis 12,Kurtosis 13,Kurtosis 14,Kurtosis 15,Kurtosis 16,Kurtosis 1,Kurtosis 2,Kurtosis 3,Kurtosis 4,Kurtosis 5,Kurtosis 6,Kurtosis 7,Kurtosis 8,Kurtosis 9,Kurtosis 10,Kurtosis 11,Kurtosis 12,Kurtosis 13,Kurtosis 14,Kurtosis 15,Kurtosis 16,"corr(1,2)","corr(1,3)","corr(2,3)","corr(1,4)","corr(2,4)","corr(3,4)","corr(1,5)","corr(2,5)","corr(3,5)","corr(4,5)","corr(1,6)","corr(2,6)","corr(3,6)","corr(4,6)","corr(5,6)","corr(1,7)","corr(2,7)","corr(3,7)","corr(4,7)","corr(5,7)","corr(6,7)","corr(1,8)","corr(2,8)","corr(3,8)","corr(4,8)","corr(5,8)","corr(6,8)","corr(7,8)","corr(1,9)","corr(2,9)","corr(3,9)","corr(4,9)","corr(5,9)","corr(6,9)","corr(7,9)","corr(8,9)","corr(1,10)","corr(2,10)","corr(3,10)","corr(4,10)","corr(5,10)","corr(6,10)","corr(7,10)","corr(8,10)","corr(9,10)","corr(1,11)","corr(2,11)","corr(3,11)","corr(4,11)","corr(5,11)","corr(6,11)","corr(7,11)","corr(8,11)","corr(9,11)","corr(10,11)","corr(1,12)","corr(2,12)","corr(3,12)","corr(4,12)","corr(5,12)","corr(6,12)","corr(7,12)","corr(8,12)","corr(9,12)","corr(10,12)","corr(11,12)","corr(1,13)","corr(2,13)","corr(3,13)","corr(4,13)","corr(5,13)","corr(6,13)","corr(7,13)","corr(8,13)","corr(9,13)","corr(10,13)","corr(11,13)","corr(12,13)","corr(1,14)","corr(2,14)","corr(3,14)","corr(4,14)","corr(5,14)","corr(6,14)","corr(7,14)","corr(8,14)","corr(9,14)","corr(10,14)","corr(11,14)","corr(12,14)","corr(13,14)","corr(1,15)","corr(2,15)","corr(3,15)","corr(4,15)","corr(5,15)","corr(6,15)","corr(7,15)","corr(8,15)","corr(9,15)","corr(10,15)","corr(11,15)","corr(12,15)","corr(13,15)","corr(14,15)","corr(1,16)","corr(2,16)","corr(3,16)","corr(4,16)","corr(5,16)","corr(6,16)","corr(7,16)","corr(8,16)","corr(9,16)","corr(10,16)","corr(11,16)","corr(12,16)","corr(13,16)","corr(14,16)","corr(15,16)",eig_1,eig_2,eig_3,eig_4,eig_5,eig_6,eig_7,eig_8,eig_9,eig_10,eig_11,eig_12,eig_13,eig_14,eig_15,eig_16,hfd1,hfd2,hfd3,hfd4,hfd5,hfd6,hfd7,hfd8,hfd9,hfd10,hfd11,hfd12,hfd13,hfd14,hfd15,hfd16,PFD 1,PFD 2,PFD 3,PFD 4,PFD 5,PFD 6,PFD 7,PFD 8,PFD 9,PFD 10,PFD 11,PFD 12,PFD 13,PFD 14,PFD 15,PFD 16,channel 1 δ(0.5–4Hz),channel 1 θ(4–7Hz),channel 1 α(8–12Hz),channel 1 β(12–30Hz),channel 1 low γ(30–70Hz),channel 1 high γ(70–180Hz),channel 2 δ(0.5–4Hz),channel 2 θ(4–7Hz),channel 2 α(8–12Hz),channel 2 β(12–30Hz),channel 2 low γ(30–70Hz),channel 2 high γ(70–180Hz),channel 3 δ(0.5–4Hz),channel 3 θ(4–7Hz),channel 3 α(8–12Hz),channel 3 β(12–30Hz),channel 3 low γ(30–70Hz),channel 3 high γ(70–180Hz),channel 4 δ(0.5–4Hz),channel 4 θ(4–7Hz),channel 4 α(8–12Hz),channel 4 β(12–30Hz),channel 4 low γ(30–70Hz),channel 4 high γ(70–180Hz),channel 5 δ(0.5–4Hz),channel 5 θ(4–7Hz),channel 5 α(8–12Hz),channel 5 β(12–30Hz),channel 5 low γ(30–70Hz),channel 5 high γ(70–180Hz),channel 6 δ(0.5–4Hz),channel 6 θ(4–7Hz),channel 6 α(8–12Hz),channel 6 β(12–30Hz),channel 6 low γ(30–70Hz),channel 6 high γ(70–180Hz),channel 7 δ(0.5–4Hz),channel 7 θ(4–7Hz),channel 7 α(8–12Hz),channel 7 β(12–30Hz),channel 7 low γ(30–70Hz),channel 7 high γ(70–180Hz),channel 8 δ(0.5–4Hz),channel 8 θ(4–7Hz),channel 8 α(8–12Hz),channel 8 β(12–30Hz),channel 8 low γ(30–70Hz),channel 8 high γ(70–180Hz),channel 9 δ(0.5–4Hz),channel 9 θ(4–7Hz),channel 9 α(8–12Hz),channel 9 β(12–30Hz),channel 9 low γ(30–70Hz),channel 9 high γ(70–180Hz),channel 10 δ(0.5–4Hz),channel 10 θ(4–7Hz),channel 10 α(8–12Hz),channel 10 β(12–30Hz),channel 10 low γ(30–70Hz),channel 10 high γ(70–180Hz),channel 11 δ(0.5–4Hz),channel 11 θ(4–7Hz),channel 11 α(8–12Hz),channel 11 β(12–30Hz),channel 11 low γ(30–70Hz),channel 11 high γ(70–180Hz),channel 12 δ(0.5–4Hz),channel 12 θ(4–7Hz),channel 12 α(8–12Hz),channel 12 β(12–30Hz),channel 12 low γ(30–70Hz),channel 12 high γ(70–180Hz

In [49]:
res2.to_csv(join('/Users/crivera5/Documents/NonIntuitProjects/Kaggle/KaggleEEG/', 'data_local', 'training.csv'), index = False)

In [50]:
df2 = pd.read_csv(join('/Users/crivera5/Documents/NonIntuitProjects/Kaggle/KaggleEEG/', 'data_local', 'training.csv'))